# MANIPULATING TABULAR DATA

## Lesson Objectives

* Review what makes a dataset tidy.
* Meet a complete set of functions for most table manipulations.
* Learn to transform datasets with split-apply-combine procedures.
* Understand the basic join operation.

## Specific Achievements

* Reshape data frames with pandas
* Summarize data by groups with pandas
* Combine multiple data frame operations with pipes
* Combine multiple data frames with “joins” (merge)

Data frames occupy a central place in Python data analysis pipelines. The panda package provide the objects and most necessary tools to subset, reformat and transform data frames. The key functions in both packages have close counterparts in SQL (Structured Query Language), which provides the added bonus of facilitating translation between python and relational databases.

https://cyberhelp.sesync.org/census-data-manipulation-in-R-lesson/



## Worksheet

## Wide to long

The pandas package’s melt function reshapes “wide” data frames into “long” ones.

In [1]:
import pandas as pd
import numpy as np
trial_df = pd.DataFrame({"block": [1,2,3],
              "drug": [0.22,0.12,0.42],
              "control": [0.58,0.98,0.19],
              "placebo": [0.31,0.47,0.40]})
trial_df.head()

,block,control,drug,placebo
0,1,0.58,0.22,0.31
1,2,0.98,0.12,0.47
2,3,0.19,0.42,0.40


In [2]:
tidy_trial_df = pd.melt(trial_df,
                  id_vars=['block'],
                  var_name='treatment',
                  value_name='response')
tidy_trial_df.head()

,block,treatment,response
0,1,control,0.58
1,2,control,0.98
2,3,control,0.19
3,1,drug,0.22
4,2,drug,0.12


## Long to wide

Use the "pivot" function to go from long format to wide.

In [3]:
df2 = tidy_trial_df.pivot(index='block',
                          columns='treatment',
                          values='response')
df2.columns

Index(['control', 'drug', 'placebo'], dtype='object', name='treatment')

In [4]:
df2.reset_index()

treatment,block,control,drug,placebo
0,1,0.58,0.22,0.31
1,2,0.98,0.12,0.47
2,3,0.19,0.42,0.40


In [5]:
df2

treatment,control,drug,placebo
block,,,
1,0.58,0.22,0.31
2,0.98,0.12,0.47
3,0.19,0.42,0.40


Consider survey data on participant’s age and income stored in a EAV structure.

In [6]:
from io import StringIO, BytesIO

text_string = StringIO("""
participant,attr,val
1,age,24
2,age,57
3,age,13
1,income,30
2,income,60
""")

survey_df = pd.read_csv(text_string, sep=",")
survey_df

,participant,attr,val
0,1,age,24
1,2,age,57
2,3,age,13
3,1,income,30
4,2,income,60


Transform the data with the pivot function, which “reverses” a melt. These are equivalent to spread and gather in the dplyr r package.

In [7]:
tidy_survey = survey_df.pivot(index='participant',
                          columns='attr',
                          values='val')
print(tidy_survey.head())

attr          age  income
participant              
1            24.0    30.0
2            57.0    60.0
3            13.0     NaN


In [8]:
tidy_survey = tidy_survey.reset_index()
tidy_survey.columns

Index(['participant', 'age', 'income'], dtype='object', name='attr')

In [12]:
tidy_survey.reset_index()

attr,index,participant,age,income
0,0,1,24.0,30.0
1,1,2,57.0,60.0
2,2,3,13.0,NaN


Note that "reset_index" adds the index as a column. It generates a new inde starting from 0 to the number of rows minus 1.

In [15]:
tidy_survey

attr,participant,age,income
0,1,24.0,30.0
1,2,57.0,60.0
2,3,13.0,NaN


## Sample Data

In [16]:
import pandas as pd
cbp = pd.read_csv('data/cbp15co.csv')
cbp.describe()

,FIPSTATE,FIPSCTY,EMP,QP1,AP,EST,N1_4,N5_9,N10_19,N20_49,...,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY
count,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,...,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06,2.126601e+06
mean,3.017996e+01,1.059362e+02,3.216730e+02,4.063474e+03,1.614668e+04,2.158477e+01,1.175502e+01,3.980026e+00,2.733421e+00,1.938450e+00,...,3.719748e-01,9.585719e-02,3.534937e-02,2.110692e-02,9.649201e-03,6.458193e-03,3.464684e-03,1.534844e-03,5.398082e+01,1.059362e+02
std,1.523679e+01,1.353254e+02,6.419011e+03,1.141995e+05,4.079178e+05,3.673087e+02,2.132928e+02,6.198514e+01,4.384888e+01,3.364651e+01,...,6.786445e+00,1.806645e+00,7.134052e-01,5.533558e-01,2.326193e-01,1.729379e-01,1.174842e-01,1.035507e-01,2.145026e+01,1.353254e+02
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.100000e+01,1.000000e+00
25%,1.800000e+01,3.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500000e+01,3.100000e+01
50%,2.900000e+01,7.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.600000e+01,7.500000e+01
75%,4.200000e+01,1.270000e+02,6.200000e+01,4.550000e+02,2.019000e+03,8.000000e+00,4.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.200000e+01,1.270000e+02
max,5.600000e+01,9.990000e+02,4.007163e+06,7.783332e+07,2.393404e+08,2.651120e+05,1.597560e+05,4.227300e+04,2.873500e+04,2.125100e+04,...,4.058000e+03,9.850000e+02,3.430000e+02,2.480000e+02,1.080000e+02,7.000000e+01,4.200000e+01,5.100000e+01,9.500000e+01,9.990000e+02


In [17]:
print(cbp.dtypes)


FIPSTATE     int64
FIPSCTY      int64
NAICS       object
EMPFLAG     object
EMP_NF      object
EMP          int64
QP1_NF      object
QP1          int64
AP_NF       object
AP           int64
EST          int64
N1_4         int64
N5_9         int64
N10_19       int64
N20_49       int64
N50_99       int64
N100_249     int64
N250_499     int64
N500_999     int64
N1000        int64
N1000_1      int64
N1000_2      int64
N1000_3      int64
N1000_4      int64
CENSTATE     int64
CENCTY       int64
dtype: object


In [18]:
import numpy as np
import pandas as pd

cbp = pd.read_csv(
  'data/cbp15co.csv',
  na_values = "NULL",
  keep_default_na=False,
  dtype =  {"FIPSTATE": np.str, 
  "FIPSCTY": np.str}
  )

In [19]:
import pandas as pd
import numpy as np
acs =  pd.read_csv(
  'data/ACS/sector_ACS_15_5YR_S2413.csv',
  dtype = {"FIPS": np.str}
  )

In [20]:
print(acs.dtypes)

FIPS              object
County            object
Sector            object
median_income    float64
dtype: object


## Typical Data Manipulation Functions

## Filter Pattern matching

In [21]:
cbp2 = cbp[cbp['NAICS'].str.contains("----")]
cbp2 = cbp2[~cbp2.NAICS.str.contains("-----")]
cbp2.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,0,63,1
10,01,001,21----,,H,82,H,713,H,3294,...,0,0,0,0,0,0,0,0,63,1
17,01,001,22----,,H,196,H,4793,H,18611,...,0,0,0,0,0,0,0,0,63,1
27,01,001,23----,,G,372,G,2891,G,13801,...,0,0,0,0,0,0,0,0,63,1
93,01,001,31----,,H,971,H,15386,H,64263,...,0,0,1,0,0,0,0,0,63,1


In [25]:
cbp3 = cbp[cbp['NAICS'].str.contains('[0-9]{2}----')]
cbp3.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,0,63,1
10,01,001,21----,,H,82,H,713,H,3294,...,0,0,0,0,0,0,0,0,63,1
17,01,001,22----,,H,196,H,4793,H,18611,...,0,0,0,0,0,0,0,0,63,1
27,01,001,23----,,G,372,G,2891,G,13801,...,0,0,0,0,0,0,0,0,63,1
93,01,001,31----,,H,971,H,15386,H,64263,...,0,0,1,0,0,0,0,0,63,1


## Altering, updating and transforming columns


In [26]:
cbp3["FIPS"] = cbp3["FIPSTATE"]+cbp3["FIPSCTY"]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
cbp3.assign(FIPS2=lambda x: x['FIPSTATE']+x['FIPSCTY'])

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY,FIPS,FIPS2
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,63,1,01001,01001
10,01,001,21----,,H,82,H,713,H,3294,...,0,0,0,0,0,0,63,1,01001,01001
17,01,001,22----,,H,196,H,4793,H,18611,...,0,0,0,0,0,0,63,1,01001,01001
27,01,001,23----,,G,372,G,2891,G,13801,...,0,0,0,0,0,0,63,1,01001,01001
93,01,001,31----,,H,971,H,15386,H,64263,...,1,0,0,0,0,0,63,1,01001,01001
163,01,001,42----,,J,211,H,2034,H,11071,...,0,0,0,0,0,0,63,1,01001,01001
218,01,001,44----,,G,2631,G,14905,G,61502,...,0,0,0,0,0,0,63,1,01001,01001
351,01,001,48----,,J,124,H,1229,H,5128,...,0,0,0,0,0,0,63,1,01001,01001
381,01,001,51----,,H,73,H,924,H,3407,...,0,0,0,0,0,0,63,1,01001,01001
401,01,001,52----,,G,375,G,4201,G,16328,...,0,0,0,0,0,0,63,1,01001,01001


In [28]:
cbp3.shape

(58901, 27)

In [29]:
cbp3.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY,FIPS
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,63,1,01001
10,01,001,21----,,H,82,H,713,H,3294,...,0,0,0,0,0,0,0,63,1,01001
17,01,001,22----,,H,196,H,4793,H,18611,...,0,0,0,0,0,0,0,63,1,01001
27,01,001,23----,,G,372,G,2891,G,13801,...,0,0,0,0,0,0,0,63,1,01001
93,01,001,31----,,H,971,H,15386,H,64263,...,0,1,0,0,0,0,0,63,1,01001


## Select

In [30]:
cbp2.columns

Index(['FIPSTATE', 'FIPSCTY', 'NAICS', 'EMPFLAG', 'EMP_NF', 'EMP', 'QP1_NF',
       'QP1', 'AP_NF', 'AP', 'EST', 'N1_4', 'N5_9', 'N10_19', 'N20_49',
       'N50_99', 'N100_249', 'N250_499', 'N500_999', 'N1000', 'N1000_1',
       'N1000_2', 'N1000_3', 'N1000_4', 'CENSTATE', 'CENCTY'],
      dtype='object')

In [31]:
cbp3 = cbp3[['FIPS','NAICS','N1_4', 'N5_9', 'N10_19']] 
cbp3.head()

,FIPS,NAICS,N1_4,N5_9,N10_19
1,01001,11----,5,1,0
10,01001,21----,0,1,1
17,01001,22----,2,1,2
27,01001,23----,51,13,7
93,01001,31----,9,4,4


In [32]:
cbp4= cbp.filter(regex='^N|FIPS|NAICS',axis=1) 
cbp4.head()

,FIPSTATE,FIPSCTY,NAICS,N1_4,N5_9,N10_19,N20_49,N50_99,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4
0,01,001,------,430,171,118,81,35,6,2,1,0,0,0,0,0
1,01,001,11----,5,1,0,0,1,0,0,0,0,0,0,0,0
2,01,001,113///,4,1,0,0,1,0,0,0,0,0,0,0,0
3,01,001,1133//,4,1,0,0,1,0,0,0,0,0,0,0,0
4,01,001,11331/,4,1,0,0,1,0,0,0,0,0,0,0,0


## Join

In [38]:
sector =  pd.read_csv(
  'data/ACS/sector_naics.csv',
  dtype = {"NAICS": np.int64})
print(sector.dtypes)

Sector    object
NAICS      int64
dtype: object


In [39]:
print(cbp.dtypes)

FIPSTATE    object
FIPSCTY     object
NAICS       object
EMPFLAG     object
EMP_NF      object
EMP          int64
QP1_NF      object
QP1          int64
AP_NF       object
AP           int64
EST          int64
N1_4         int64
N5_9         int64
N10_19       int64
N20_49       int64
N50_99       int64
N100_249     int64
N250_499     int64
N500_999     int64
N1000        int64
N1000_1      int64
N1000_2      int64
N1000_3      int64
N1000_4      int64
CENSTATE     int64
CENCTY       int64
dtype: object


In [40]:
cbp.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY
0,01,001,------,,G,10454,G,76437,G,321433,...,6,2,1,0,0,0,0,0,63,1
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,0,63,1
2,01,001,113///,,H,70,H,790,H,3551,...,0,0,0,0,0,0,0,0,63,1
3,01,001,1133//,,H,70,H,790,H,3551,...,0,0,0,0,0,0,0,0,63,1
4,01,001,11331/,,H,70,H,790,H,3551,...,0,0,0,0,0,0,0,0,63,1


In [41]:
print(sector.dtypes)
print(sector.shape) #24 economic sectors
sector.head()

Sector    object
NAICS      int64
dtype: object
(24, 2)


,Sector,NAICS
0,agriculture forestry fishing and hunting,11
1,mining quarrying and oil and gas extraction,21
2,utilities,22
3,construction,23
4,manufacturing,31


## Many-to-One

In [42]:
logical_idx = cbp['NAICS'].str.match('[0-9]{2}----') #boolean index
cbp = cbp.loc[logical_idx]
cbp.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY
1,01,001,11----,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,0,63,1
10,01,001,21----,,H,82,H,713,H,3294,...,0,0,0,0,0,0,0,0,63,1
17,01,001,22----,,H,196,H,4793,H,18611,...,0,0,0,0,0,0,0,0,63,1
27,01,001,23----,,G,372,G,2891,G,13801,...,0,0,0,0,0,0,0,0,63,1
93,01,001,31----,,H,971,H,15386,H,64263,...,0,0,1,0,0,0,0,0,63,1


In [43]:
cbp.shape

(58901, 26)

In [44]:
cbp['NAICS']= cbp.NAICS.apply(lambda x: np.int64(x[0:2])) # select first two digits

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
#Many to one to join economic sector code to NAICS
cbp_test = cbp.merge(sector, on = "NAICS", how='inner')
cbp_test.head()

,FIPSTATE,FIPSCTY,NAICS,EMPFLAG,EMP_NF,EMP,QP1_NF,QP1,AP_NF,AP,...,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,CENSTATE,CENCTY,Sector
0,01,001,11,,H,70,H,790,H,3566,...,0,0,0,0,0,0,0,63,1,agriculture forestry fishing and hunting
1,01,003,11,,G,14,H,362,H,1035,...,0,0,0,0,0,0,0,63,3,agriculture forestry fishing and hunting
2,01,005,11,,G,180,G,1472,G,5909,...,0,0,0,0,0,0,0,63,5,agriculture forestry fishing and hunting
3,01,007,11,,H,70,H,527,H,2499,...,0,0,0,0,0,0,0,63,7,agriculture forestry fishing and hunting
4,01,009,11,,J,10,J,43,J,219,...,0,0,0,0,0,0,0,63,9,agriculture forestry fishing and hunting


In [46]:
print(cbp_test.shape)

(56704, 27)


## Group By

In [47]:
cbp["FIPS"] = cbp["FIPSTATE"]+cbp["FIPSCTY"]
cbp = cbp.merge(sector, on = "NAICS")

cbp_grouped = cbp.groupby(['FIPS','Sector'])
cbp_grouped

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
cbp_grouped.dtypes

FIPSTATE FIPSCTY  \
FIPS  Sector                                                                
01001 accommodation and food services                      object  object   
      administrative and support and waste management...   object  object   
      agriculture forestry fishing and hunting             object  object   
      arts entertainment and recreation                    object  object   
      construction                                         object  object   
...                                                           ...     ...   
56999 professional scientific and technical services       object  object   
      retail trade                                         object  object   
      transportation and warehousing                       object  object   
      utilities                                            object  object   
      wholesale trade                                      object  object   

                                                          NAICS EMPFLAG  \
FIPS  Sector                                                              
01001 accommodation and food services                     int64  object   
      administrative and support and waste management...  int64  object   
      agriculture forestry fishing and hunting            int64  object   
      arts entertainment and recreation                   int64  object   
      construction                                        int64  object   
...                                                         ...     ...   
56999 professional scientific and technical services      int64  object   
      retail trade                                        int64  object   
      transportation and warehousing                      int64  object   
      utilities                                           int64  object   
      wholesale trade                                     int64  object   

                                                          EMP_NF    EMP  \
FIPS  Sector                                                              
01001 accommodation and food services                     object  int64   
      administrative and support and waste management...  object  int64   
      agriculture forestry fishing and hunting            object  int64   
      arts entertainment and recreation                   object  int64   
      construction                                        object  int64   
...                                                          ...    ...   
56999 professional scientific and technical services      object  int64   
      retail trade                                        object  int64   
      transportation and warehousing                      object  int64   
      utilities                                           object  int64   
      wholesale trade                                     object  int64   

                                                          QP1_NF    QP1  \
FIPS  Sector                                                              
01001 accommodation and food services                     object  int64   
      administrative and support and waste management...  object  int64   
      agriculture forestry fishing and hunting            object  int64   
      arts entertainment and recreation                   object  int64   
      construction                                        object  int64   
...                                                          ...    ...   
56999 professional scientific and technical services      object  int64   
      retail trade                                        object  int64   
      transportation and warehousing                      object  int64   
      utilities                                           object  int64   
      wholesale trade                                     object  int64   

                                                           AP_NF     AP  ...  \
FIPS  Sector                                   

## Summarize

In [49]:
grouped_df = (cbp
.groupby(['FIPS', 'Sector']) 
.agg('sum')
.filter(regex='^N')
.drop(columns=['NAICS'])
)

grouped_df.head(5)

N1_4  N5_9  N10_19  \
FIPS  Sector                                                                   
01001 accommodation and food services                       23     8      19   
      administrative and support and waste management...    18     2       3   
      agriculture forestry fishing and hunting               5     1       0   
      arts entertainment and recreation                      5     2       1   
      construction                                          51    13       7   

                                                          N20_49  N50_99  \
FIPS  Sector                                                               
01001 accommodation and food services                         29      12   
      administrative and support and waste management...       2       0   
      agriculture forestry fishing and hunting                 0       1   
      arts entertainment and recreation                        2       0   
      construction                                             4       0   

                                                          N100_249  N250_499  \
FIPS  Sector                                                                   
01001 accommodation and food services                            0         0   
      administrative and support and waste management...         0         0   
      agriculture forestry fishing and hunting                   0         0   
      arts entertainment and recreation                          1         0   
      construction                                               0         0   

                                                          N500_999  N1000  \
FIPS  Sector                                                                
01001 accommodation and food services                            0      0   
      administrative and support and waste management...         0      0   
      agriculture forestry fishing and hunting                   0      0   
      arts entertainment and recreation                          0      0   
      construction                                               0      0   

                                                          N1000_1  N1000_2  \
FIPS  Sector                                                                 
01001 accommodation and food services                           0        0   
      administrative and support and waste management...        0        0   
      agriculture forestry fishing and hunting                  0        0   
      arts entertainment and recreation                         0        0   
      construction                                              0        0   

                                                          N1000_3  N1000_4  
FIPS  Sector                                                                
01001 accommodation and food services                           0        0  
      administrative and support and waste management...        0        0  
      agriculture forestry fishing and hunting                  0        0  
      arts entertainment and recreation                         0        0  
      construction                                              0        0

In [50]:
print(grouped_df.shape)

(56704, 13)


In [51]:
print(acs.shape)

(59698, 4)


In [52]:
acs_cbp = grouped_df.merge(acs,on='FIPS',)
print(acs_cbp.shape)

(1061416, 17)


In [53]:
acs_cbp.head()

,FIPS,N1_4,N5_9,N10_19,N20_49,N50_99,N100_249,N250_499,N500_999,N1000,N1000_1,N1000_2,N1000_3,N1000_4,County,Sector,median_income
0,01001,23,8,19,29,12,0,0,0,0,0,0,0,0,"Autauga County, Alabama",agriculture forestry fishing and hunting,27235.0
1,01001,23,8,19,29,12,0,0,0,0,0,0,0,0,"Autauga County, Alabama",mining quarrying and oil and gas extraction,102722.0
2,01001,23,8,19,29,12,0,0,0,0,0,0,0,0,"Autauga County, Alabama",construction,31632.0
3,01001,23,8,19,29,12,0,0,0,0,0,0,0,0,"Autauga County, Alabama",manufacturing,40233.0
4,01001,23,8,19,29,12,0,0,0,0,0,0,0,0,"Autauga County, Alabama",wholesale trade,41656.0
